# 使用dlib的correlation_tracker和face_recognition
# 追蹤特定一張臉_同時進行人臉辨識
# GUI: Tkinter

In [ ]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk, Image
import numpy as np
import face_recognition
import random
import time
import dlib
# 上一個影幀出現的臉名稱
last_face_names = []

# 追蹤所使用的顏色
rectangleColor = (0,165,255) 

# 創建一個tracker，這邊只是for找Zonejia
tracker = dlib.correlation_tracker()

# 當trackingFace = 1時，就代表需要使用tracker找東西
trackingFace = 0

# 預先載入加分&扣分物的圖片
obj_width = 40

garbage_img = cv2.imread('apple.jpeg')
garbage_img = cv2.resize(garbage_img,(40,40),interpolation=cv2.INTER_CUBIC)
garbage_rows, garbage_cols, channels = garbage_img.shape

present_img = cv2.imread('bad.png')
present_img = cv2.resize(present_img,(40,40),interpolation=cv2.INTER_CUBIC)
present_rows, present_cols, channels = present_img.shape

print(garbage_rows, garbage_cols, channels)
print(present_rows, present_cols, channels)


In [ ]:
root = tk.Tk()

# Load a sample picture and learn how to recognize it.
obama_image = face_recognition.load_image_file("obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]
zonejia_image = face_recognition.load_image_file("zonejia_image.jpg")
zonejia_face_encoding = face_recognition.face_encodings(zonejia_image)[0]

# 當等於True才處理該Frame，False則忽略
process_this_frame = True

# 單純攝影機設定
width, height = 1000, 750
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
face_landmarks_list = []
# ------------------遊戲初始區----------------------
# 初始化 加分掉落物
init_y = 5
garbage_location_x = np.random.randint(low=4, high=width-obj_width, size=5)
garbage_location_y = np.zeros(5)*10
# 初始化 扣分掉落物
present_location_x = np.random.randint(low=4, high=width-obj_width, size=5)
present_location_y = np.zeros(5)*10

# 倒數計時出現的加分數字(位置)
bonus_position = (0,0) # (x, y)

# 每隔10毫秒掉落距離
garbage_speed = 5
# 分數
score = 0
# ------------------遊戲初始區----------------------

tStart = -1
tEnd = -1
finalCount = 0
def peace_close():
    global root, cap
#     root.quit()
    root.destroy() 
    cap.release()
    
root.bind('<Escape>', lambda e: peace_close)
lmain = tk.Label(root)
lmain.pack()

close_btn = tk.Button(root, text='點一下關閉', command=peace_close)
close_btn.pack()

def show_frame():
    # Initialize some variables
    global face_locations, face_encodings, face_names
#     global last_face_locations, last_face_encodings, last_face_names
    global tracker, trackingFace

    global process_this_frame
    global garbage_location_x, garbage_location_y
    global present_location_x, present_location_y

    global score, tStart, tEnd, bonus_position, finalCount
    _, frame = cap.read()
#     frame = cv2.flip(frame, 1) # 將照片左右轉向一下
    
    
    
    # 畫圖開始
#     small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    
    # 計時
    if tStart < 0:
        tStart = time.time()
    tEnd = time.time()
    finalSecond = int(tEnd - tStart)

    print('目前時間：', finalSecond)
    
    # 使用dlib.tracker時就不在乎，是否要跳Frame處理
    if trackingFace:
        print('tracking中')
        # Update the tracker and request information about the
        # quality of the tracking update
        trackingQuality = tracker.update(frame)

        # If the tracking quality is good enough, determine the
        # updated position of the tracked region and draw the
        # rectangle
        # trackingQuality越高，代表越嚴格的追蹤(太不像就會放棄追蹤)
#         if trackingQuality >= 8.75:
#         if trackingQuality >= 6.75: 
#         if trackingQuality >= 5.75: 
        if trackingQuality >= 3.75:
#         if trackingQuality >= 2.75:    
            print('tracking到好目標，繼續tracking')
            tracked_position = tracker.get_position()

            t_x = int(tracked_position.left())
            t_y = int(tracked_position.top())
            t_w = int(tracked_position.width())
            t_h = int(tracked_position.height())
            cv2.rectangle(frame, (t_x, t_y),
                            (t_x + t_w , t_y + t_h),
                            rectangleColor ,2)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, '@Zone-Jia@', (t_x + 6, t_y+t_h - 6), font, 1.0, (255, 255, 255), 1)
      
        else:
            # If the quality of the tracking update is not
            # sufficient (e.g. the tracked region moved out of the
            # screen) we stop the tracking of the face and in the
            # next loop we will find the largest face in the image
            # again
            print('tracking不到好目標，設定tarckingFace=0')
            trackingFace = 0

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)

        face_names = []

        # 找出所有臉中是否有符合 '歐巴馬臉'或'宗嘉臉'的
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            match = face_recognition.compare_faces([obama_face_encoding, zonejia_face_encoding], face_encoding)
            name = "Unknown"

            if match[0]:
                name = "Barack"
            if match[1]:
                name = 'Zone-Jia'
            face_names.append(name)

        # 超級嘗試Coding 起點

        # 判斷是否要執行dlib的tracker動作
        # 如果上一次有找到宗嘉，這次則沒有用face_recognition找到的話：

        if 'Zone-Jia' in face_names:
            zonejia_face_index = face_names.index('Zone-Jia')

            # face, and we determine this based on the largest
            # area of the found rectangle. First initialize the
            # required variables to 0
            maxArea = 0
            x = 0
            y = 0
            w = 0
            h = 0

            # Loop over all faces and check if the area for this
            # face is the largest so far
            # We need to convert it to int here because of the
            # requirement of the dlib tracker. If we omit the cast to
            # int here, you will get cast errors since the detector
            # returns numpy.int32 and the tracker requires an int
            (_top, _right, _bottom, _left) = face_locations[zonejia_face_index]

            _top *= 2
            _right *= 2
            _bottom *= 2
            _left *= 2
            print('_top:', _top)
            print('_right:', _right)
            print('_bottom:', _bottom)
            print('_left:', _left)
            if (_bottom - _top) * (_right - _left) > maxArea:
                maxArea = (_bottom - _top) * (_right - _left)
                print('maxArea:', maxArea)
            # If one or more faces are found, initialize the tracker
            # on the largest face in the picture
            if maxArea > 0:

                # Initialize the tracker
                tracker.start_track(frame,
                                    dlib.rectangle(_left - 10,
                                                   _top - 20,
                                                   _right + 10,
                                                   _bottom+ 20))

                # Set the indicator variable such that we know the
                # tracker is tracking a region in the image
                trackingFace = 1

            if trackingFace == 1:
                print('已經準備好tracking了')
            else:
                print('並未準備好tracking')
                
    # 匡出 被找到的臉部

    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if 'Zone-Jia' == name:
            tracker.start_track(frame,
                                    dlib.rectangle(2*left - 10,
                                                   2*top - 20,
                                                   2*right + 10,
                                                   2*bottom+ 20))
            continue
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
    #         top *= 4
    #         right *= 4
    #         bottom *= 4
    #         left *= 4
        top *= 2
        right *= 2
        bottom *= 2
        left *= 2
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        
    # 超級嘗試Coding 底部

    process_this_frame = not process_this_frame

    
    # 畫圖結束
    
    
    cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    lmain.imgtk = imgtk
    lmain.configure(image=imgtk)
    lmain.after(10, show_frame)

show_frame()
root.mainloop()

cap.release()
print('和平結束')

In [ ]:
# 緊急停止方案
peace_close()